In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
df_w = pd.read_csv('Pi data/Logged_Data_WaterTemps/2023_5_8_10_54_27/data_water_temps.csv')
df_w.head(10)

,time,temp1,temp2,temp3
0,2023-05-08 10:54:27.817300,3.69,5.31,3.44
1,2023-05-08 10:54:27.818114,3.69,5.25,3.44
2,2023-05-08 10:54:27.818871,3.75,5.25,3.44
3,2023-05-08 10:54:27.819660,3.75,5.31,3.44
4,2023-05-08 10:54:30.200286,3.75,5.25,3.44
5,2023-05-08 10:54:31.940981,3.75,5.31,3.44
6,2023-05-08 10:54:33.677580,3.75,5.31,3.44
7,2023-05-08 10:54:35.418445,3.75,5.31,3.44
8,2023-05-08 10:54:37.154940,3.69,5.31,3.44
9,2023-05-08 10:54:38.895562,3.75,5.31,3.44


In [1]:
# remove rows with extreme values
df_w = df_w[df_w['temp1'] < 100]
df_w = df_w[df_w['temp1'] > -100]
df_w = df_w[df_w['temp2'] < 100]
df_w = df_w[df_w['temp2'] > -100]
df_w = df_w[df_w['temp3'] < 100]
df_w = df_w[df_w['temp3'] > -100]

df_w['Time'] = pd.to_datetime(df_w['time']).dt.round('min')

df_w['Water'] = df_w['temp1']
df_w.drop(columns=['time', 'temp1', 'temp2', 'temp3'], inplace=True)

# take the mean of different Water temperatures to get a single value for each minute
df_w = df_w.groupby('Time').mean()

display(df_w.head(10))

NameError: name 'df_w' is not defined

In [4]:
df_fd = pd.read_csv('Data/Freezerdata mei 07-10.csv')

# get mean from refrigerated space 1 and 2
df_fd['Fridge'] = df_fd[['Refrigerated space (1)', 'Refrigerated space (2)']].mean(axis=1)

# convert time to datetime
df_fd['Time'] = pd.to_datetime(df_fd['Date']).dt.round('min')

# 1 is door open, 0 if closed
df_fd.loc[df_fd.Door == 'Closed', 'Door'] = 0
df_fd.loc[df_fd.Door == 'Open', 'Door'] = 1
df_fd['Door'] = df_fd['Door'].astype('float64')

# drop old columns
df_fd.drop(columns=['Date', 'Refrigerated space (1)', 'Refrigerated space (2)'], inplace=True)

df_fd = df_fd.groupby('Time').mean()

# use linear interpolation to fill missing values
df_fd = df_fd.interpolate(method='linear')

display(df_fd.head(10))

,Hot gas pipe,Liquid pipe,Suction pipe,Energie meter (kwh / pulse) Watt,Door,Fridge
Time,,,,,,
2023-05-07 00:01:00,25.9,24.7,16.8,NaN,NaN,3.50
2023-05-07 00:02:00,29.0,26.1,16.2,100.0,0.0,0.35
2023-05-07 00:03:00,38.5,27.3,11.2,800.0,0.0,-1.45
2023-05-07 00:04:00,40.6,26.9,10.1,0.0,0.0,0.20
2023-05-07 00:05:00,38.6,26.4,10.5,100.0,0.0,0.70
2023-05-07 00:06:00,35.8,26.0,11.2,50.0,0.0,1.15
2023-05-07 00:07:00,33.4,25.7,12.1,100.0,0.0,1.55
2023-05-07 00:08:00,31.5,25.6,12.8,50.0,0.0,1.90
2023-05-07 00:09:00,30.1,25.5,13.6,100.0,0.0,2.15


In [5]:
# merge df and df_fd using Time 
merged = pd.merge(df_w, df_fd, on='Time', how='inner')

# add time as column instead of index
merged['Time'] = merged.index
cols = list(merged.columns)
cols = [cols[-1]] + cols[:-1]
merged = merged[cols]
merged.reset_index(drop=True, inplace=True)

merged.head(display())

,Time,Water,Hot gas pipe,Liquid pipe,Suction pipe,Energie meter (kwh / pulse) Watt,Door,Fridge
0,2023-05-08 10:54:00,3.720000,24.7,24.9,19.8,0.0,0.0,6.75
1,2023-05-08 10:55:00,3.743143,24.6,24.8,20.2,0.0,0.0,6.65
2,2023-05-08 10:56:00,3.750000,24.3,24.7,20.3,50.0,0.0,6.80
3,2023-05-08 10:57:00,3.750000,24.3,24.7,20.4,0.0,0.0,6.75
4,2023-05-08 10:58:00,3.750000,24.1,24.2,20.3,50.0,0.0,6.75
...,...,...,...,...,...,...,...,...
2851,2023-05-10 10:49:00,5.799412,74.1,35.6,9.7,700.0,1.0,2.50
2852,2023-05-10 10:50:00,5.810000,74.4,35.4,9.8,750.0,1.0,3.05
2853,2023-05-10 10:51:00,5.814118,74.5,34.7,10.5,650.0,1.0,2.75
2854,2023-05-10 10:52:00,5.848000,74.4,34.8,10.1,800.0,1.0,2.25


In [6]:
fig = px.line(merged, x='Time', y=['Water', 'Hot gas pipe', 'Liquid pipe', 'Suction pipe',
       'Door', 'Fridge'], title='Time Series with Rangeslider Small Vat')
fig.update_xaxes(rangeslider_visible=True)
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [7]:
# prepare data for Neural Network
data = merged[['Water', 'Hot gas pipe', 'Liquid pipe', 'Suction pipe',
       'Energie meter (kwh / pulse) Watt', 'Door', 'Fridge']]

# add columns for previous values
data['Water-1'] = data['Water'].shift(1)
data['Water-2'] = data['Water'].shift(2)
data['Water-3'] = data['Water'].shift(3)

# set Water-1 at index 1 as Water
data.loc[0, 'Water-1'] = data.loc[0, 'Water']

# set Water-2 at index 2 as Water-1
data.loc[0:1, 'Water-2'] = data.loc[1, 'Water-1']

# set Water-3 at index 3 as Water-3
data.loc[0:2, 'Water-3'] = data.loc[2, 'Water-2']

# drop rows with NaN values
data.dropna(inplace=True)

# set index as time
data['TIme'] = data.index

# add x and y
data['x'] = 29
data['y'] = 16

data.head(display())

,Water,Hot gas pipe,Liquid pipe,Suction pipe,Energie meter (kwh / pulse) Watt,Door,Fridge,Water-1,Water-2,Water-3,TIme,x,y
0,3.720000,24.7,24.9,19.8,0.0,0.0,6.75,3.720000,3.720000,3.720000,0,29,16
1,3.743143,24.6,24.8,20.2,0.0,0.0,6.65,3.720000,3.720000,3.720000,1,29,16
2,3.750000,24.3,24.7,20.3,50.0,0.0,6.80,3.743143,3.720000,3.720000,2,29,16
3,3.750000,24.3,24.7,20.4,0.0,0.0,6.75,3.750000,3.743143,3.720000,3,29,16
4,3.750000,24.1,24.2,20.3,50.0,0.0,6.75,3.750000,3.750000,3.743143,4,29,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2851,5.799412,74.1,35.6,9.7,700.0,1.0,2.50,5.763714,5.750000,5.746571,2851,29,16
2852,5.810000,74.4,35.4,9.8,750.0,1.0,3.05,5.799412,5.763714,5.750000,2852,29,16
2853,5.814118,74.5,34.7,10.5,650.0,1.0,2.75,5.810000,5.799412,5.763714,2853,29,16
2854,5.848000,74.4,34.8,10.1,800.0,1.0,2.25,5.814118,5.810000,5.799412,2854,29,16


In [8]:
# split data in train and test
train = data.iloc[:int(len(data)*0.8)]
test = data.iloc[int(len(data)*0.8):]

In [9]:
# write data to csv
train.to_csv('Data/train.csv', index=False)
test.to_csv('Data/test.csv', index=False)